In [226]:
%run Comment_Questions.ipynb # Exectute temporary training Parameters

# NEU Unit (Reconfiguration Map)

## Initialize Modules

###### Imports

In [227]:
# Deep Learning & ML
import tensorflow as tf
import keras as Kr
from keras import backend as K
from keras.models import Sequential
from keras import layers
from keras import utils as np_utils

# General
import numpy as np
import time

# Message Boxes
import tkinter
from tkinter import messagebox

### Define Reconfiguration Map

1. First applies the map $\psi(x)\triangleq e^{\frac1{1-|x|}}I_{\{|x|<1\}}$ component-wise.

2. Applies the diagonalization map to that output: $ \left(x_1,\dots,x_d\right)\mapsto
                \begin{pmatrix}
                x_1 & & 0\\
                &\ddots &\\
                0 & & x_d\\
                \end{pmatrix}$
3. Adds trainable $d\times d$ (weight) matrix $A$ and trainable bias (vector) $b \in \mathbb{R}^d$ to that output:

### Build Reconfiguration

In [318]:
import tensorflow as tf
print('TensorFlow:', tf.__version__)

class BiasLayer(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super(BiasLayer, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.weight = self.add_weight(shape=[input_shape[1],input_shape[1]], # CURRENTLY NOT A MATRIX FIX THIS!!!
                                    initializer='zeros',
                                    trainable=True)
        self.bias = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        self.location = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        self.scale = self.add_weight(shape=input_shape[1:],
                                    initializer='zeros',
                                    trainable=True)
        
        
        
    def call(self, x):
        # Shift and scale data
        #---------------------#
        # Shift 
        x_shift = x - self.location
        # Rescale
        x_mod = tf.math.multiply(x_shift,self.scale)
        
        # Apply bumpy function & tangent space transformation
        #----------------------------------------------------#
        # 1) Bump Function
        x_in_abs = tf.math.abs(x_mod)
        x_in_abs = tf.constant([1,2,.1])
        Logic_x_leq1 = tf.math.sign(tf.keras.activations.relu(1-x_in_abs)) # Takes value 1 iff |x|<=1 else 0: since probability of |x|=1 is 0 we should be ok
        x_thresheld = Logic_x_leq1*tf.math.exp(-1/(1-tf.math.pow(x_in_abs,-1))) # Computes bump function at thresholds with previous logic
        # 2) Map to Diagonal Matrix & Apply Bias
        x_out = tf.expand_dims((x_thresheld + self.bias), 1) 
        # 3) Apply weights
        x_out = (x_out * self.weight) 
        # 4) Apply Matrix Exponential
        x_out = tf.linalg.expm(x_out)
        
        # Mulpliply Transformation with shifted data
        #--------------------------------------------#
        x_out = tf.linalg.matvec(x_out,x_shift)
        
        # Recenter Data
        #---------------#
        x_out = x_out + self.location
        
        # Return Ouput
        return x_out

input_layer = tf.keras.Input(shape=[5])
x  = BiasLayer()(input_layer)
model = tf.keras.Model(inputs=[input_layer], outputs=[x])

model.summary()

TensorFlow: 2.1.0


ValueError: in converted code:

    <ipython-input-318-e401b8a28fc6>:40 call  *
        x_out = tf.expand_dims((x_thresheld + self.bias), 1)
    /scratch/users/kratsioa/.local/lib/python3.7/site-packages/tensorflow_core/python/ops/math_ops.py:915 binary_op_wrapper
        return func(x, y, name=name)
    /scratch/users/kratsioa/.local/lib/python3.7/site-packages/tensorflow_core/python/ops/math_ops.py:1194 _add_dispatch
        return gen_math_ops.add_v2(x, y, name=name)
    /scratch/users/kratsioa/.local/lib/python3.7/site-packages/tensorflow_core/python/ops/gen_math_ops.py:483 add_v2
        "AddV2", x=x, y=y, name=name)
    /scratch/users/kratsioa/.local/lib/python3.7/site-packages/tensorflow_core/python/framework/op_def_library.py:742 _apply_op_helper
        attrs=attr_protos, op_def=op_def)
    /scratch/users/kratsioa/.local/lib/python3.7/site-packages/tensorflow_core/python/framework/func_graph.py:595 _create_op_internal
        compute_device)
    /scratch/users/kratsioa/.local/